In [1]:
import joblib
import numpy as np

# Carregar o modelo e os scalers
model_filename = 'mlp_model.pkl'
scalerx_filename = 'scaler_x.pkl'
scalery_filename = 'scaler_y.pkl'

loaded_model = joblib.load(model_filename)
scalerx = joblib.load(scalerx_filename)
scalery = joblib.load(scalery_filename)

## valores para Reynolds e Stdp
Re = np.linspace(1000, 8000, 100)  
Stdp = np.linspace(0.2992, 0.4331, 100)  

# Empacotar os valores de Re e Stdp para passar ao modelo
X = np.array(list(zip(Re, Stdp)))

# Escalar os dados
X_scaled = scalerx.transform(X)

# Fazer a previsão com o modelo
f = loaded_model.predict(X_scaled).reshape(-1, 1)

# Desescalar a previsão
f_inverse = scalery.inverse_transform(f)

c:\Users\Carlos\anaconda3\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator MLPRegressor from version 1.3.0 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\Carlos\anaconda3\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.3.0 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [3]:
import numpy as np
import plotly.graph_objs as go
import plotly.offline as pyo




# Criar uma grade de Re e Stdp (Assumindo que Re e Stdp já estão definidos)
Re_grid, Stdp_grid = np.meshgrid(Re, Stdp)

# Escalar os pontos da grade
grid_points = np.c_[Re_grid.ravel(), Stdp_grid.ravel()]
grid_points_scaled = scalerx.transform(grid_points)

# Fazer previsões para cada ponto da grade usando o modelo carregado
f_grid_pred = loaded_model.predict(grid_points_scaled).reshape(-1, 1)

# Desescalar as previsões
f_pred_grid = scalery.inverse_transform(f_grid_pred)

# Ajustar a forma de f_pred_grid para que corresponda à forma de Re_grid e Stdp_grid
f_pred_grid = f_pred_grid.reshape(Re_grid.shape)

# Criar a superfície com os resultados da ANN
surface_ann = go.Surface(
    x=Re_grid, y=Stdp_grid, z=f_pred_grid, 
    colorscale='Viridis',
    name='Superfície ANN',
    colorbar=dict(title='ANN f', len=0.5, x=0.9),
    contours={
        "z": {
            "show": True,
            "start": np.min(f_pred_grid),
            "end": np.max(f_pred_grid),
            "size": (np.max(f_pred_grid) - np.min(f_pred_grid)) / 20,  # Ajustar o espaçamento dos contornos
            "color": "black",
        }
    }
)

# Definir o layout do gráfico
layout = go.Layout(
    title='Superfície de Resultados da ANN e da Fórmula',
    scene=dict(
        xaxis=dict(title='Reynolds Number (Re)', backgroundcolor='white', gridcolor='lightgrey'),
        yaxis=dict(title='Stdp', backgroundcolor='white', gridcolor='lightgrey'),
        zaxis=dict(title='f', backgroundcolor='white', gridcolor='lightgrey'),
        bgcolor='white'
    ),
    paper_bgcolor='white',
    plot_bgcolor='white',
    showlegend=True
)

# Criar a figura com ambas as superfícies (ANN e fórmula)
fig = go.Figure(data=[surface_ann], layout=layout)

# Mostrar o gráfico
pyo.plot(fig, auto_open=True)


'temp-plot.html'